## Lab 2: the OpenAI Python Client Library 

In [1]:
from dotenv import load_dotenv
from openai import OpenAI
from IPython.display import display, Markdown

### <span style="color: green;">Question: what is a Client Library?</span>

In [2]:
load_dotenv(override=True)

True

In [3]:
openai = OpenAI()

In [4]:
message = "Tell me a fun quick fact."
messages = [{"role": "user", "content": message}]

In [5]:
response = openai.chat.completions.create(
    model="gpt-4.1-nano",
    messages=messages,
    temperature=0.5,
    max_tokens=50)

reply = response.choices[0].message.content
display(Markdown(reply))

Did you know that honey never spoils? Archaeologists have found pots of honey in ancient Egyptian tombs that are over 3,000 years old and still perfectly edible!

In [ ]:
reply = response.choices[0].message.content
display(Markdown(reply))

### <span style="color: orange;">Question: what is the Chat Completions API? What's the Responses API? Why use one over the other?</span>

Gemini:

That's right! You don't have to manually worry about the full URL of the endpoint when using the official client library like openai = OpenAI(). 🥳

The reason you don't need to specify the full URL, like "https://api.openai.com/v1/chat/completions", is because the client library handles that for you.

How the Client Library Manages the URL
The official client library is structured to automatically build the correct URL for every API call:

1. Base URL is Hardcoded
The main OpenAI class (OpenAI()) has the base URL hardcoded internally: https://api.openai.com/v1. This is the consistent starting point for almost all OpenAI services.

2. Method Maps to Path
The methods you call on the client object directly map to the API path:

Client Method Call	Maps to URL Path
openai.chat.completions.create(...)	Appends the path /chat/completions
openai.images.generate(...)	Appends the path /images/generations
openai.models.list()	Appends the path /models

Export to Sheets
When you execute a method like openai.chat.completions.create(), the client automatically combines the hardcoded base URL with the method's corresponding path to construct the full, correct endpoint URL:

Full Endpoint URL=Base URL+API Path
Full Endpoint URL=‘https://api.openai.com/v1‘ + ‘/chat/completions‘
This abstraction means you focus only on what you want to do (e.g., create a chat completion), and the client library handles the low-level details of where to send the request.

The Chat Completions API and the Responses API are the two primary ways to interact with OpenAI's large language models. The Responses API is a newer, more capable evolution of the Chat Completions API.


## 1. Chat Completions API (/v1/chat/completions)
What it Is: The original and industry-standard endpoint for text generation (what most LLMs use).

Core Feature: It is stateless. Every request is treated independently, and you must manually manage the entire conversation history (the list of system, user, and assistant messages) and send it with every new prompt.

Data Structure: Input and output are formatted as a simple array of messages.

## 2. Responses API (/v1/responses)
What it Is: The recommended, newer endpoint designed for building complex, agentic applications. It bundles the capabilities of the old Chat Completions and Assistants APIs into a single interface.

Core Features:

Stateful/Simplified Context: You can easily chain turns by referencing a previous_response_id or setting store: true, allowing the service to manage the conversation history for you.

Agentic by Default: It natively supports complex, multi-step reasoning where the model can autonomously use tools or call functions multiple times in a single turn.

Built-in Tools: It integrates powerful capabilities like web search and file search without requiring you to write custom orchestration code.

Data Structure: Input and output use a more flexible concept called Items and Output to represent text, tool calls, and other complex actions.

## 1. Chat Completions API Example (Stateless, Message-Array Input) 💾
The Chat Completions API is characterized by its messages array, where you must send the entire conversation history (system, user, and assistant turns) in every request to maintain context.

In [ ]:
from openai import OpenAI
# The client will automatically look for the OPENAI_API_KEY environment variable.
client = OpenAI()

# 1. Define the conversation history (System + User message for the first turn)
messages = [
    {"role": "system", "content": "You are a witty, helpful assistant who loves classic literature."},
    {"role": "user", "content": "What is the main theme of Moby Dick?"}
]

# 2. Call the chat.completions.create endpoint
response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=messages
)

# 3. Extract the text
reply = response.choices[0].message.content
print("Chat Completions Reply:", reply)

# ----------------------------------------------------------------------------------
# To continue the conversation (2nd turn):
# You must manually append the assistant's previous reply and the new user message.
messages.append({"role": "assistant", "content": reply})
messages.append({"role": "user", "content": "That's a good summary. Tell me about the author."})

# Call the API again with the *full, updated* messages list
response_2 = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=messages
)

print("\nSecond Turn Reply:", response_2.choices[0].message.content)

## 2. Responses API Example (Stateful, Simplified Input) 🤖
The Responses API simplifies single-turn requests with the input parameter and can manage state using previous_response_id for multi-turn conversations. It also makes tools easier to use.

A. Simple Single-Turn Request

In [ ]:
from openai import OpenAI
client = OpenAI()

# 1. Simple, direct text input
response = client.responses.create(
    model="gpt-4o",
    instructions="You are a professional copywriter.",
    input="Create a compelling tagline for a new brand of smart coffee mugs."
)

# 2. Extract the text (the output structure is different)
reply = response.output_text
print("Responses API Reply:", reply)

B. Stateful Multi-Turn Request
Here, the API manages the context of the joke for the follow-up question, even without passing the full message history:

In [ ]:
# First turn (tell a joke)
response_1 = client.responses.create(
    model="gpt-4o",
    input="tell me a joke about Python programming."
)
print("Joke:", response_1.output_text)

# Second turn (follow-up question using the previous response's ID)
# The API uses the 'id' to recall the joke and maintain context.
response_2 = client.responses.create(
    model="gpt-4o",
    previous_response_id=response_1.id,
    input="Now, explain the technical concept that makes the joke funny."
)
print("\nExplanation:", response_2.output_text)